In [71]:
import pandas as pd
import numpy as np
from siuba import *
from calitp import *

In [72]:
import altair as alt

In [73]:
from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [74]:
pd.set_option("display.precision", 2)

In [95]:
df = pd.read_excel('OFFICIAGrantsOverviewandWalkthrough.xlsx')

In [96]:
df = to_snakecase(df)

In [97]:
df = df.rename(columns={'grant_type__formula,_competitive,_both,_discretionary_n_a_':'distribution_type',
                       'current__in_this_fy__or_expired':'current'})

In [98]:
df.sample()

,grant_#,division,grant_program,notice_of_funding,fy_due_date,months_to_apply__c_d_,fy_cycle,funding_amount_available,agency_eligibility,grant_type,...,eligibility:_school_district,eligibility:_mpo,eligibility:_county_regional_govt__not_mpo,eligibility:_other_local_entity__non_municipality_,eligibility:_state_public_lands,eligibility:_federal_public_lands,eligibility:_tribal_government,eligibility:_non_profit,eligibility:_state_department_,eligibility:_for_profit_companies
11,12.0,SB1,​​Solutions for Congested Corridors Program,2022-08-17 00:00:00,2022-12-02 00:00:00,3,2 Year,2.50e+08,RTPA or county transportation commission or au...,Competitive,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 25 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   grant_#                                             76 non-null     float64
 1   division                                            77 non-null     object 
 2   grant_program                                       77 non-null     object 
 3   notice_of_funding                                   77 non-null     object 
 4   fy_due_date                                         77 non-null     object 
 5   months_to_apply__c_d_                               77 non-null     object 
 6   fy_cycle                                            77 non-null     object 
 7   funding_amount_available                            77 non-null     float64
 8   agency_eligibility                                  77 non-null     object 
 9   g

In [100]:
df.federal_or_state_funded.value_counts()

Federal    60
State      12
Both        4
Missing     1
Name: federal_or_state_funded, dtype: int64

In [101]:
(
    df
    >>filter(_.federal_or_state_funded=='State')
    >> group_by(_.division, _.federal_or_state_funded)
    >> summarize(avg_funds=(_.funding_amount_available.mean()))
)

,division,federal_or_state_funded,avg_funds
0,Clean CA,State,2.96e+08
1,DLA,State,1.27e+07
2,DRMT,State,9.16e+07
3,SB1,State,4.78e+08
4,SB1/DLA,State,5.79e+08


In [102]:
chart = (
    alt.Chart(
        (
    df
    >>filter(_.federal_or_state_funded=='State')
    >> group_by(_.division, _.federal_or_state_funded)
    >> summarize(avg_funds=(_.funding_amount_available.mean()))
)
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "federal_or_state_funded",
            title="Origin of Funding",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Type")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [103]:
chart = (
    alt.Chart(
       (
    df
    >>filter(_.federal_or_state_funded=='Federal')
    >> group_by(_.division, _.federal_or_state_funded)
    >> summarize(avg_funds=(_.funding_amount_available.mean()))
)
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "federal_or_state_funded",
            title="Origin of Funding",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Type (Nation-wide)")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [104]:
df.sample(10)

,grant_#,division,grant_program,notice_of_funding,fy_due_date,months_to_apply__c_d_,fy_cycle,funding_amount_available,agency_eligibility,grant_type,...,eligibility:_school_district,eligibility:_mpo,eligibility:_county_regional_govt__not_mpo,eligibility:_other_local_entity__non_municipality_,eligibility:_state_public_lands,eligibility:_federal_public_lands,eligibility:_tribal_government,eligibility:_non_profit,eligibility:_state_department_,eligibility:_for_profit_companies
75,76.0,Federal Transit Administration (DOT),Enhancing Mobility Innovation,2021-11-12 00:00:00,2022-01-11 00:00:00,1,Missing,2.00e+06,"Providers of public transportation, including ...",Competitive,...,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0
12,13.0,SB1,Trade Corridor Enhancement Program,2022-08-17 00:00:00,2022-11-18 00:00:00,3,2 Years,8.00e+08,"Caltrans, Regional Agencies, Counties, Cities,...",Competitive,...,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN
52,53.0,Federal Transit Administration (DOT),Ferry Service for Rural Communities,2nd Quarter of 2022,Missing,Missing,TBD,2.00e+09,States,Grant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
10,11.0,DRMT,Low Carbon Transit Operations Program (already...,2022-02-01 00:00:00,2022-03-25 00:00:00,1,1 Year,7.81e+07,Entities in the list provided by the SCO and a...,Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,14.0,SB1,Local Streets and Roads Program,Missing,2022-07-01 00:00:00,Missing,1 Year,1.20e+09,"City, County",Competitive,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,75.0,Federal Transit Administration (DOT),Bus Exportable Power Systems,2022-04-12 00:00:00,2022-06-13 00:00:00,2,Missing,1.00e+06,"Providers of public transportation, including ...",Competitive,...,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0
62,63.0,Federal Transit Administration (DOT),Public Transportation on Indian Reservations C...,1st Quarter of 2022,Missing,Missing,Year of allocation plus two years,4.58e+07,Federally recognized Indian Tribes and Alaska ...,Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
49,50.0,Federal Highway Administration (DOT),Highway Research & Development Program,TBD,Missing,Missing,Available until expended,3.10e+08,Varies,Grant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,32.0,DLA,High Risk Rural Road Eligible Projects in HSIP,Missing,Missing,Missing,Missing,0.00e+00,Missing,Missing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,36.0,Federal Highway Administration (DOT),Bridge Investment Program,2022-06-10 00:00:00,2022-09-08 00:00:00,2,1 Year,2.50e+09,•State\n •MPO (w/ pop. >200K)\n• Local governm...,Competitive,...,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN


In [105]:
df>>group_by(_.division)>>summarize(avg_funds = (_.funding_amount_available.mean()))

,division,avg_funds
0,Caltrans - Unknown Division,2.30e+09
1,Clean CA,2.96e+08
2,DLA,1.14e+08
3,DLA,0.00e+00
4,DRMT,1.30e+09
5,Federal Highway Administration (DOT),4.22e+08
6,Federal Railroad Administration (DOT),1.00e+09
7,Federal Transit Administration (DOT),5.40e+08
8,Office of the Secretary (DOT),2.07e+09
9,SB1,8.56e+08


In [108]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='State')
            >> group_by(_.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "division",
            title="Division",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Division")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [109]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='Federal')
            >> group_by(_.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "division",
            title="Division",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Division (Nation-wide)")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [110]:
df>>group_by(_.grant_type, _.division)>>summarize(avg_funds = (_.funding_amount_available.mean()))

,grant_type,division,avg_funds
0,Both,DRMT,6.04e+08
1,Both,SB1,1.17e+08
2,Competitive,Caltrans - Unknown Division,2.30e+09
3,Competitive,DLA,1.35e+08
4,Competitive,DRMT,2.50e+09
5,Competitive,Federal Highway Administration (DOT),4.63e+08
6,Competitive,Federal Railroad Administration (DOT),1.00e+09
7,Competitive,Federal Transit Administration (DOT),4.48e+08
8,Competitive,Office of the Secretary (DOT),2.44e+09
9,Competitive,SB1,1.72e+09


In [111]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='State')
            >> group_by(_.grant_type, _.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds",
            axis=alt.Axis(format="$.2s"),
            title=("Average Funds Available"),
            sort=("-y"),
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "grant_type",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
            legend=alt.Legend(title=("Grant Type")),
        ),
    )
    .properties(title="Average Funds Available by Disvision and Grant Distribution Type")
)
chart = styleguide.preset_chart_config(chart)
chart

alt.Chart(...)

In [112]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='Federal')
            >> group_by(_.grant_type, _.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds",
            axis=alt.Axis(format="$.2s"),
            title=("Average Funds Available"),
            sort=("-y"),
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "grant_type",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
            legend=alt.Legend(title=("Grant Type")),
        ),
    )
    .properties(title="Average Funds Available by Disvision and Grant Distribution Type (Nation-wide)")
)
chart = styleguide.preset_chart_config(chart)
chart

alt.Chart(...)